# **Modelling and Evaluation**

## Objectives

*   Fit and evaluate a regression model to predict the Sale Price of houses in Ames, Iowa.


## Inputs

* outputs/datasets/collection/house_prices_records.csv
* Instructions on which variables to use for data cleaning and feature engineering. They are found in each respective notebook.

## Outputs

* Train set (features and target)
* Test set (features and target)
* Data cleaning and Feature Engineering pipeline
* Modeling pipeline
* Feature importance plot

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Step 1 - Load Data

In [ ]:
import numpy as np
import pandas as pd

# Import raw data path
df = (pd.read_csv("outputs/datasets/collection/house_prices_records.csv").drop(labels=['EnclosedPorch', 'WoodDeckSF'], axis=1))

# Print shape and first 5 rows of data
print(df.shape)
df.head(5)

In [ ]:
df.info()

---

# Step 2 - ML Pipeline with all data

## Create ML Pipeline

Custom transformation class created to transform float type to integer type, which can then be applied into the ML Pipeline.

Information on how to create was taken from the following sources:
- [Andrew Villazon](https://www.andrewvillazon.com/custom-scikit-learn-transformers/)
- [SciKit Learn - Transformer Mixin](https://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html)
- [SciKit Learn - Base Estimater](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


# Create own class to transform float to integer
class FloatIntegerConversionTransformer(BaseEstimator, TransformerMixin):
    def __init__(self,columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X[self.columns] = X[self.columns].astype(int)
        return X


# Create custom class to transform missing data fo '2ndFlrSF'
class Fill_Missing_2ndFlrSF_Transformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['2ndFlrSF'] = X.apply(
            lambda row: row['GrLivArea'] - row['1stFlrSF'] if pd.isna(row['2ndFlrSF']) else row['2ndFlrSF'], axis=1
        )
        return X


# Create custom class to transform missing data fo 'BsmtFinType1'
class Fill_Missing_BsmtFinType1_Transformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['BsmtFinType1'] = X.apply(
           lambda row: 1 if pd.isna(row['BsmtFinType1']) and row['TotalBsmtSF'] > 0 else (0 if pd.isna(row['BsmtFinType1']) else row['BsmtFinType1']), axis=1
        )
        return X


# Create custom class to transform missing data fo 'BsmtFinType1'
class Fill_Missing_GarageFinish_Transformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['GarageFinish'] = X.apply(
           lambda row: 1 if pd.isna(row['GarageFinish']) and row['GarageArea'] > 0 else (0 if pd.isna(row['GarageFinish']) else row['GarageFinish']), axis=1
        )
        return X


# Create custom class to transform missing data fo 'GarageYrBlt'
class Fill_Missing_GarageYrBlt_Transformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['GarageYrBlt'] = X.apply(
           lambda row: '0' if pd.isna(row['GarageYrBlt']) and row['GarageArea'] < 0 else (row['YearBuilt'] if pd.isna(row['GarageYrBlt']) else row['GarageYrBlt']), axis=1
        )
        return X


# Create custom class to create new feature 'TotalLivArea'
class Create_TotalLivArea_Transformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['TotalLivArea'] = X['GrLivArea'] + X['TotalBsmtSF']
        return X


In [ ]:
from sklearn.pipeline import Pipeline

# Feature Engineering
from feature_engine.imputation import MeanMedianImputer, ArbitraryNumberImputer
from feature_engine.transformation import LogTransformer
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.outliers import Winsorizer
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine import transformation as vt

# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor




def PipelineOptimization(model):
    pipeline_base = Pipeline([
        # Data cleaning
        ('fill_missing_2ndflrsf', Fill_Missing_2ndFlrSF_Transformer()),
        ('median_imputer', MeanMedianImputer(
            imputation_method='median', variables=['BedroomAbvGr'])),
        ('fill_missing_bsmtfintype1', Fill_Missing_BsmtFinType1_Transformer()),
        ('fill_missing_garagefinish', Fill_Missing_GarageFinish_Transformer()),
        ('fill_missing_garageyrblt', Fill_Missing_GarageYrBlt_Transformer()),
        ('mean_imputer',  MeanMedianImputer(
            imputation_method='mean', variables=['LotFrontage'])),
        ('arbitrary_number_imputer', ArbitraryNumberImputer(
            arbitrary_number=0, variables=['MasVnrArea'])),
        ('create_totallivarea', Create_TotalLivArea_Transformer()),
        ('float_to_integer', FloatIntegerConversionTransformer(['2ndFlrSF','BedroomAbvGr','BsmtFinType1','GarageFinish','GarageYrBlt','LotFrontage','MasVnrArea'])),

        # Feature engineering
        ('log', vt.LogTransformer(variables=['GrLivArea'],base='e')),
        ('yj', vt.YeoJohnsonTransformer(variables=['1stFlrSF','TotalBsmtSF','YearBuilt','TotalLivArea'])),
        ('winsorizer_iqr', Winsorizer(capping_method='iqr', fold=1.5, tail='both', variables=['1stFlrSF','TotalBsmtSF','TotalLivArea','OverallQual','GarageArea','GrLivArea'])),
        ("SmartCorrelatedSelection", SmartCorrelatedSelection(variables=None, method="spearman", threshold=0.8, selection_method="variance")),

    
        ("feat_scaling", StandardScaler()),

        ("feat_selection",  SelectFromModel(model)),

        ("model", model),
    ])

    return pipeline_base


## Custom Class for hyperparameter optimisation

Custom class from Code Institute ML Walkthrough project 2.

In [ ]:
from sklearn.model_selection import GridSearchCV


class HyperparameterOptimizationSearch:

    def __init__(self, models, params):
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv, n_jobs, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f"\nRunning GridSearchCV for {key} \n")
            model = PipelineOptimization(self.models[key])

            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score',
                   'mean_score', 'max_score', 'std_score']
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns], self.grid_searches

## Split Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['SalePrice'], axis=1),
    df['SalePrice'],
    test_size=0.2,
    random_state=0
)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

## Grid Search CV - Sklearn

### Use standard hyperparameters to find most suitable algorithm 

In [ ]:
models_quick_search = {
    'LinearRegression': LinearRegression(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
    "RandomForestRegressor": RandomForestRegressor(random_state=0),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
    "AdaBoostRegressor": AdaBoostRegressor(random_state=0),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=0),
    "XGBRegressor": XGBRegressor(random_state=0),
}

params_quick_search = {
    'LinearRegression': {},
    "DecisionTreeRegressor": {},
    "RandomForestRegressor": {},
    "ExtraTreesRegressor": {},
    "AdaBoostRegressor": {},
    "GradientBoostingRegressor": {},
    "XGBRegressor": {},
}

Quick GridSearch CV

In [ ]:
search = HyperparameterOptimizationSearch(models=models_quick_search, params=params_quick_search)
search.fit(X_train, y_train,
           scoring='r2',
           n_jobs=-1, cv=5)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary 

### Extensive Search

Documentation on [ExtraTreesRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.ExtraTreeRegressor.html#extratreeregressor) can be found here.

In [ ]:
models_search = {
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=0),
}

params_search = {
    "ExtraTreesRegressor":{
        'model__n_estimators': [100, 200, 300],  # Number of trees in the forest
        'model__max_depth': [None, 10, 20, 30], # The maximum depth of the tree
        'model__min_samples_split': [2, 5, 10], # The minimum number of samples required to split an internal node
        'model__min_samples_leaf': [1, 2, 4], # The minimum number of samples required to be at a leaf node
        'model__min_weight_fraction_leaf': [0.0], # The minimum weighted fraction of the sum total of weights
        'model__max_features': [1.0, None], # The number of features to consider when looking for the best split
    }
}

In [ ]:
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train,scoring='r2',n_jobs=-1, cv=5)

### Results from extensive GridSearchCV

Following the extensive search on hyperparameters. The R2 value has improved from 0.829443 to 0.844095

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary 

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

In [ ]:
grid_search_pipelines[best_model].best_params_

In [ ]:
best_regressor_pipeline = grid_search_pipelines[best_model].best_estimator_
best_regressor_pipeline

## Assess Feature Importance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# after data cleaning and feature engineering, the features may have changes
data_cleaning_feat_eng_steps = 13
columns_after_data_cleaning_feat_eng = (Pipeline(best_regressor_pipeline.steps[:data_cleaning_feat_eng_steps])
                                        .transform(X_train)
                                        .columns)

best_features = columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support(
)].to_list()

# create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng[best_regressor_pipeline['feat_selection'].get_support()],
    'Importance': best_regressor_pipeline['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in descending order. "
      f"The model was trained on them: \n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(kind='bar', color='green', x='Feature', y='Importance')
plt.show()

In [ ]:
best_features

TotalLivArea is a calculated field and requires transformation from the original dataset.
The two features that are required are: ['GrLivArea','TotalBsmtSF'].

We will update best features to:

['KitchenQual', 'OverallQual', 'YearBuilt', 'GrLivArea', 'TotalBsmtSF']

Will amend the pipeline to calculate the TotalLivArea then drop the two features.

In [ ]:
best_features = ['KitchenQual', 'OverallQual', 'YearBuilt', 'GrLivArea', 'TotalBsmtSF']
best_features

## Evaluate on Train and Test Sets

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error 
import numpy as np


def regression_performance(X_train, y_train, X_test, y_test,pipeline):
  """
  # Gets train/test sets and pipeline and evaluates the performance
  - for each set (train and test) call regression_evaluation()
  which will evaluate the pipeline performance
  """

  print("Model Evaluation \n")
  print("* Train Set")
  regression_evaluation(X_train,y_train,pipeline)
  print("* Test Set")
  regression_evaluation(X_test,y_test,pipeline)



def regression_evaluation(X,y,pipeline):
  """
  # Gets features and target (either from train or test set) and pipeline
  - it predicts using the pipeline and the features
  - calculates performance metrics comparing the prediction to the target
  """
  prediction = pipeline.predict(X)
  print('R2 Score:', r2_score(y, prediction).round(3))  
  print('Mean Absolute Error:', mean_absolute_error(y, prediction).round(3))  
  print('Mean Squared Error:', mean_squared_error(y, prediction).round(3))  
  print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y, prediction)).round(3))
  print("\n")

  

def regression_evaluation_plots(X_train, y_train, X_test, y_test,pipeline, alpha_scatter=0.5):
  """
  # Gets Train and Test set (features and target), pipeline, and adjust dots transparency 
  at scatter plot
  - It predicts on train and test set
  - It creates Actual vs Prediction scatterplots, for train and test set
  - It draws a red diagonal line. In theory, a good regressor should predict
  close to the actual, meaning the dot should be close to the diagonal red line
  The closer the dots are to the line, the better

  """
  pred_train = pipeline.predict(X_train)
  pred_test = pipeline.predict(X_test)


  fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,6))
  sns.scatterplot(x=y_train , y=pred_train, alpha=alpha_scatter, ax=axes[0])
  sns.lineplot(x=y_train , y=y_train, color='red', ax=axes[0])
  axes[0].set_xlabel("Actual")
  axes[0].set_ylabel("Predictions")
  axes[0].set_title("Train Set")

  sns.scatterplot(x=y_test , y=pred_test, alpha=alpha_scatter, ax=axes[1])
  sns.lineplot(x=y_test , y=y_test, color='red', ax=axes[1])
  axes[1].set_xlabel("Actual")
  axes[1].set_ylabel("Predictions")
  axes[1].set_title("Test Set")

  plt.show()

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, best_regressor_pipeline)
regression_evaluation_plots(X_train, y_train, X_test, y_test, best_regressor_pipeline)

### Performance evaluation

- Both the train and test set return a R2 value which is > 0.7 as part of the business requirements
- The R2 value in the test set is lower, but within acceptable ranges.
- There may be a suggestion that the model has overfitted in the train set and futher exploration may be warrented in the future to improves the models performance on unseen data.

---

# Refit ML pipeline with best features

The following features were identifies as the best features and are to be refitted into the Pipeline - ['KitchenQual', 'OverallQual', 'YearBuilt', 'GrLivArea', 'TotalBsmtSF']

We will run the data through the pipeline which will create the 'TotalLivArea' feature and then drop the 'GrLivArea', 'TotalBsmtSF'

In [ ]:
from sklearn.pipeline import Pipeline

# Feature Engineering
from transformers import Drop_Cols_Transformer, Create_TotalLivArea_Transformer
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.outliers import Winsorizer
from feature_engine.selection import SmartCorrelatedSelection
from feature_engine import transformation as vt

# Feat Scaling
from sklearn.preprocessing import StandardScaler

# Feat Selection
from sklearn.feature_selection import SelectFromModel

# ML algorithms
from sklearn.ensemble import ExtraTreesRegressor

# Pipeline Optmization: Model
def PipelineOptimization(model):
    pipeline = Pipeline([
        # Data cleaning
        ('create_totallivarea', Create_TotalLivArea_Transformer()),
        ('drop_columns', Drop_Cols_Transformer(columns=['GrLivArea', 'TotalBsmtSF'])),

        # Feature engineering
        ('yj', vt.YeoJohnsonTransformer(variables=['YearBuilt','TotalLivArea'])),
        ('winsorizer_iqr', Winsorizer(capping_method='iqr', fold=1.5, tail='both', variables=['TotalLivArea','OverallQual'])),
        
        ("scaler", StandardScaler()),
        ('model', ExtraTreesRegressor(max_features=1.0, min_samples_leaf=4, min_samples_split=10, random_state=0))
    ])

    return pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['SalePrice'], axis=1),
    df['SalePrice'],
    test_size=0.2,
    random_state=0
)

print("* Train set:", X_train.shape, y_train.shape,
      "\n* Test set:",  X_test.shape, y_test.shape)

In [ ]:
X_train = X_train.filter(best_features)
X_test = X_test.filter(best_features)

print("* Train set:", X_train.shape, y_train.shape, "\n* Test set:",  X_test.shape, y_test.shape)
X_train.head(3)

In [ ]:
models_search

In [ ]:
best_parameters = grid_search_pipelines[best_model].best_params_
best_parameters

In [ ]:
models_search = {
    'ExtraTreesRegressor': ExtraTreesRegressor(random_state=0),
}

params_search = {
    'ExtraTreesRegressor': {
        'model__max_depth': [None],
        'model__max_features': [1.0],
        'model__min_samples_leaf': [4],
        'model__min_samples_split': [10],
        'model__min_weight_fraction_leaf': [0.0],
        'model__n_estimators': [100],
    },
}

In [ ]:
from sklearn.metrics import make_scorer, recall_score
search = HyperparameterOptimizationSearch(models=models_search, params=params_search)
search.fit(X_train, y_train, scoring='r2', n_jobs=-1, cv=5)

In [ ]:
grid_search_summary, grid_search_pipelines = search.score_summary(sort_by='mean_score')
grid_search_summary

In [ ]:
best_model = grid_search_summary.iloc[0, 0]
best_model

In [ ]:
pipeline_reg = grid_search_pipelines[best_model].best_estimator_
pipeline_reg

In [ ]:
regression_performance(X_train, y_train, X_test, y_test, pipeline_reg)
regression_evaluation_plots(X_train, y_train, X_test, y_test, pipeline_reg)

### Complete pipeline after selecting best features

In [ ]:
data_cleaning_feat_eng_steps = 4
columns_after_data_cleaning_feat_eng = (Pipeline(pipeline_reg.steps[:data_cleaning_feat_eng_steps]).transform(X_train).columns)

best_features = columns_after_data_cleaning_feat_eng

# Create DataFrame to display feature importance
df_feature_importance = (pd.DataFrame(data={
    'Feature': columns_after_data_cleaning_feat_eng,
    'Importance': pipeline_reg['model'].feature_importances_})
    .sort_values(by='Importance', ascending=False)
)

# Most important features statement and plot
print(f"* These are the {len(best_features)} most important features in  "
      f"descending order. The model was trained on them: "
      f"\n{df_feature_importance['Feature'].to_list()}")

df_feature_importance.plot(
    kind='bar', color='green', x='Feature', y='Importance')
plt.show()

---

# Push files to Repo

We will generate the following files

* Train set
* Test set
* Modeling pipeline
* Features importance plot

In [ ]:
import joblib
import os

version = 'v1'
file_path = f'outputs/ml_pipeline/predict_sale_price/{version}'

try:
  os.makedirs(name=file_path)
except Exception as e:
  print(e)

## Train Set: features and target

In [ ]:
X_train.info()
X_train.head(3)

In [ ]:
X_train.to_csv(f"{file_path}/X_train.csv", index=False)

In [ ]:
y_train

In [ ]:
y_train.to_csv(f"{file_path}/y_train.csv", index=False)

## Test Set: features and target

In [ ]:
X_test.info()
X_test.head(3)

In [ ]:
X_test.to_csv(f"{file_path}/X_test.csv", index=False)

In [ ]:
y_test

In [ ]:
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

## Modelling Pipeline

In [ ]:
pipeline_reg

In [ ]:
joblib.dump(value=pipeline_reg, filename=f"{file_path}/predict_house_sales_pipeline.pkl")

## Feature Importance Plot

In [ ]:
df_feature_importance.plot(kind='bar', color='green', x='Feature', y='Importance')
plt.show()

In [ ]:
df_feature_importance.plot(kind='bar', color='green', x='Feature', y='Importance')
plt.savefig(f'{file_path}/features_importance.png', bbox_inches='tight')